In [ ]:
!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1
!pip install -q pandas numpy transformers sentence-transformers ogb torch-geometric tqdm matplotlib scikit-learn

In [ ]:

import os, json, time
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from ogb.nodeproppred import PygNodePropPredDataset

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    MarianMTModel,
    MarianTokenizer,
)
from sentence_transformers import SentenceTransformer


In [ ]:
os.makedirs("data/ogbn-arxiv/raw", exist_ok=True)

if not os.path.exists("data/ogbn-arxiv/raw/titleabs.tsv"):
    !wget -q -O data/ogbn-arxiv/raw/titleabs.tsv https://snap.stanford.edu/ogb/data/misc/ogbn_arxiv/titleabs.tsv


In [ ]:
dataset = PygNodePropPredDataset(name="ogbn-arxiv", root="data/ogbn-arxiv")
data = dataset[0]

num_nodes = data.num_nodes
num_classes = int(data.y.max().item() + 1)

print(data)

In [ ]:
split_idx = dataset.get_idx_split()
train_idx = split_idx["train"]
val_idx   = split_idx["valid"]
test_idx  = split_idx["test"]


In [ ]:
mapping_file = "data/ogbn-arxiv/ogbn_arxiv/mapping/nodeidx2paperid.csv.gz"
ogb_map = pd.read_csv(mapping_file)
ogb_map.columns = ["ogb_id", "mag_id"]

raw_texts = pd.read_csv(
    "data/ogbn-arxiv/raw/titleabs.tsv",
    sep="\t",
    header=None,
    names=["mag_id", "title", "abstract"],
    on_bad_lines="skip"
)

merged = pd.merge(ogb_map, raw_texts, on="mag_id", how="left")
merged = merged.sort_values("ogb_id")

merged["full_text"] = merged["title"].fillna("") + " " + merged["abstract"].fillna("")
texts = merged["full_text"].tolist()

assert len(texts) == num_nodes
print("Loaded texts:", len(texts))


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data.edge_index = data.edge_index.to(device)
data.y = data.y.squeeze().to(device)


In [ ]:
encoder = SentenceTransformer("all-mpnet-base-v2", device=device)
encoder.eval()

emb_path = "baseline_embeddings_mpnet.pt"

print("Computing MPNet embeddings")

batch_size = 64
all_embeddings = []

for i in tqdm(range(0, len(texts), batch_size)):
    batch_texts = texts[i:i + batch_size]
    with torch.no_grad():
        emb = encoder.encode(
            batch_texts,
            convert_to_tensor=True,
            show_progress_bar=False
        )
    all_embeddings.append(emb.cpu())

node_embeddings = torch.cat(all_embeddings, dim=0)

# Save clean tensor
torch.save(node_embeddings, emb_path)

print("Saved embeddings to:", emb_path)
print("Embedding shape:", node_embeddings.shape)


In [ ]:
import torch

obj = torch.load("mpnet_model.pt", map_location="cpu")

print(type(obj))

if isinstance(obj, torch.Tensor):
    print("Tensor shape:", obj.shape)

elif isinstance(obj, dict):
    print("Dict keys:", obj.keys())
    for k, v in obj.items():
        if torch.is_tensor(v):
            print(f"{k}: tensor {tuple(v.shape)}")
        else:
            print(f"{k}: {type(v)}")


In [ ]:

SUBSET_SIZE = 1000
test_subset = test_idx[:SUBSET_SIZE].cpu()
print("Using test subset size:", len(test_subset))


In [ ]:

RESULTS_PATH = "paraphrase_attack_results_mpnet.json"

if os.path.exists(RESULTS_PATH):
    with open(RESULTS_PATH, "r") as f:
        results = json.load(f)
else:
    results = {}

results["test_subset"] = test_subset.tolist()
with open(RESULTS_PATH, "w") as f:
    json.dump(results, f, indent=2)


In [ ]:
import torch
from torch_geometric.nn import GraphSAGE

class SAGEModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes):
        super().__init__()
        self.sage_model = GraphSAGE(
            in_channels=in_channels,
            hidden_channels=hidden_channels,
            num_layers=2,
            out_channels=num_classes,
            dropout=0.5,
            act='relu'
        )

    def forward(self, x, edge_index):
        return self.sage_model(x, edge_index)

model = SAGEModel(
    in_channels=node_embeddings.size(1),
    hidden_channels=128,
    num_classes=int(data.y.max().item() + 1)
).to(device)

state_dict = torch.load("mpnet_model.pt", map_location=device)
model.load_state_dict(state_dict)
model.eval()

print("GraphSAGE model loaded successfully")


In [ ]:

@torch.no_grad()
def get_preds():
    model.eval()
    logits = model(data.x, data.edge_index)
    return logits.argmax(dim=1).cpu()

data.x = node_embeddings.clone().to(device)
baseline_preds = get_preds()

baseline_test_acc = (
    baseline_preds[test_subset] == data.y.cpu()[test_subset]
).float().mean().item()

results["baseline"] = {"test_acc": baseline_test_acc}
with open(RESULTS_PATH, "w") as f:
    json.dump(results, f, indent=2)

print("Baseline test accuracy (MPNet):", baseline_test_acc)


In [ ]:

def save_results(name, test_acc, flip_rate, cos_dist):
    results[name] = {
        "test_acc": float(test_acc),
        "accuracy_drop": float(baseline_test_acc - test_acc),
        "flip_rate": float(flip_rate),
        "cosine_mean": float(cos_dist.mean().item()),
        "cosine_median": float(cos_dist.median().item()),
        "cosine_max": float(cos_dist.max().item()),
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    }
    with open(RESULTS_PATH, "w") as f:
        json.dump(results, f, indent=2)


In [ ]:

PARA_MODEL = "t5-base"

t5_tokenizer = AutoTokenizer.from_pretrained(PARA_MODEL)
t5_model = AutoModelForSeq2SeqLM.from_pretrained(
    PARA_MODEL,
    use_safetensors=True
).to(device)
t5_model.eval()


In [ ]:

@torch.no_grad()
def paraphrase_once(text):
    inputs = t5_tokenizer(
        "paraphrase: " + text,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    ).to(device)

    outputs = t5_model.generate(
        **inputs,
        max_length=256,
        do_sample=True,
        temperature=1.3,
        top_p=0.95,
        num_beams=4
    )
    return t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

def paraphrase_two(text):
    return paraphrase_once(paraphrase_once(text))


In [ ]:

BT_LANGS = {
    "zh": ("Helsinki-NLP/opus-mt-en-zh", "Helsinki-NLP/opus-mt-zh-en"),
    "hi": ("Helsinki-NLP/opus-mt-en-hi", "Helsinki-NLP/opus-mt-hi-en"),
    "de": ("Helsinki-NLP/opus-mt-en-de", "Helsinki-NLP/opus-mt-de-en"),
}

TRANSLATORS = {}

def load_translator(name):
    tok = MarianTokenizer.from_pretrained(name)
    mdl = MarianMTModel.from_pretrained(
        name,
        weights_only=False
    ).to(device)
    mdl.eval()
    return tok, mdl

for lang, (en_x, x_en) in BT_LANGS.items():
    TRANSLATORS[lang] = {
        "en_x": load_translator(en_x),
        "x_en": load_translator(x_en),
    }


In [ ]:

@torch.no_grad()
def translate(text, tok, mdl):
    inputs = tok(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    out = mdl.generate(**inputs, num_beams=4)
    return tok.decode(out[0], skip_special_tokens=True)

def backtranslate_once(text, lang):
    tok1, mdl1 = TRANSLATORS[lang]["en_x"]
    tok2, mdl2 = TRANSLATORS[lang]["x_en"]
    return translate(translate(text, tok1, mdl1), tok2, mdl2)

def backtranslate_two(text, lang):
    return backtranslate_once(backtranslate_once(text, lang), lang)


In [ ]:

def run_attack(name, attack_fn):
    print(f"Running {name}")
    data.x = node_embeddings.clone().to(device)

    attacked = texts.copy()
    for idx in tqdm(test_subset, desc=name):
        attacked[idx] = attack_fn(texts[idx])

    embs = []
    for i in range(0, len(texts), 64):
        with torch.no_grad():
            embs.append(encoder.encode(attacked[i:i+64], convert_to_tensor=True))

    data.x = torch.cat(embs).to(device)

    preds = get_preds()
    test_preds = preds[test_subset]
    base_preds = baseline_preds[test_subset]

    test_acc = (test_preds == data.y.cpu()[test_subset]).float().mean().item()
    flip_rate = (test_preds != base_preds).float().mean().item()

    cos_dist = 1 - F.cosine_similarity(
        node_embeddings[test_subset].cpu(),
        data.x[test_subset].cpu(),
        dim=1
    )

    save_results(name, test_acc, flip_rate, cos_dist)

    torch.save(data.x[test_subset].cpu(), f"embeddings_mpnet_{name}.pt")


In [ ]:

#run_attack("paraphrase_1step", paraphrase_once)
#run_attack("paraphrase_2step", paraphrase_two)

for lang in ["zh", "hi", "de"]:
    run_attack(f"bt_{lang}_1step", lambda t, l=lang: backtranslate_once(t, l))

print("All MPNet attacks complete")


In [ ]:

with open(RESULTS_PATH, "r") as f:
    results = json.load(f)

rows = []
for k, v in results.items():
    if k in ["baseline", "test_subset"]:
        continue
    rows.append({
        "Attack": k,
        "Test Accuracy": v["test_acc"],
        "Accuracy Drop": v["accuracy_drop"],
        "Flip Rate": v["flip_rate"],
        "Cosine Mean": v["cosine_mean"],
    })

df = pd.DataFrame(rows).sort_values("Accuracy Drop", ascending=False)
df
